In [55]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [56]:
#  Import and read the charity_data.csv.

application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [57]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'ORGANIZATION', 'USE_CASE'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,1,0,5000,1
1,T3,Independent,C2000,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,1,0,5000,0
3,T3,CompanySponsored,C2000,1,10000-24999,6692,1
4,T3,Independent,C1000,1,100000-499999,142590,1


In [58]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
INCOME_AMT           9
dtype: int64

In [59]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df.APPLICATION_TYPE.value_counts()
app_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [60]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type[app_type < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [61]:
# Look at CLASSIFICATION value counts for binning
class_type = application_df.CLASSIFICATION.value_counts()
class_type

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1580        1
C2570        1
C1728        1
C4200        1
C2190        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [62]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_type[class_type < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [63]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [64]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,CLASSIFICATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [65]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [66]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values
y = y.reshape(-1, 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# # Optimization Attemp 1 - removed features and changed activation function

In [68]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=80, activation="sigmoid", input_dim=number_input_features))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 80)                2640      
_________________________________________________________________
dense_11 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 31        
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [70]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn_2.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5937 - accuracy: 0.7055
Epoch 2/100
804/804 [==============================] - 1s 973us/step - loss: 0.5747 - accuracy: 0.7209
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5713 - accuracy: 0.7226
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5689 - accuracy: 0.7252
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5669 - accuracy: 0.7256: 0s - loss: 0.5675 - accu

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 1s 888us/step - loss: 0.5662 - accuracy: 0.7268
Epoch 7/100
804/804 [==============================] - 1s 915us/step - loss: 0.5657 - accuracy: 0.7274
Epoch 8/100
804/804 [==============================] - 1s 985us/step - loss: 0.5652 - accuracy: 0.7271
Epoch 9/100
804/804 [==============================] - 1s 873us/step - loss: 0.5643 - accuracy: 0.72

804/804 [==============================] - 1s 899us/step - loss: 0.5580 - accuracy: 0.72990s - loss: 0.5602 - accuracy:  - ETA: 0s - loss: 0.5588 - accuracy: 0.
Epoch 57/100
804/804 [==============================] - 1s 980us/step - loss: 0.5579 - accuracy: 0.7303
Epoch 58/100
804/804 [==============================] - 1s 955us/step - loss: 0.5577 - accuracy: 0.73020s - loss: 0.5
Epoch 59/100
804/804 [==============================] - 1s 929us/step - loss: 0.5578 - accuracy: 0.73040s - loss: 0.5547 - accura
Epoch 60/100
804/804 [==============================] - 1s 878us/step - loss: 0.5578 - accuracy: 0.7304

Epoch 00060: saving model to checkpoints\weights.60.hdf5
Epoch 61/100
804/804 [==============================] - 1s 963us/step - loss: 0.5577 - accuracy: 0.7304
Epoch 62/100
804/804 [==============================] - 1s 905us/step - loss: 0.5576 - accuracy: 0.7299
Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7299
Epoch 64/100
80

In [72]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5645 - accuracy: 0.7235
Loss: 0.564540445804596, Accuracy: 0.723498523235321


In [75]:
nn_2.save("AlphabetSoupCharity_Optimization1.h5")

# # Optimization Attempt  2 - add hidden layers and neurons

In [73]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=50, activation="relu"))

# third hidden layer
nn_3.add(tf.keras.layers.Dense(units=20, activation="relu"))

# fourth hidden layer
nn_3.add(tf.keras.layers.Dense(units=10, activation="relu"))

# fifth hidden layer
nn_3.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 80)                2640      
_________________________________________________________________
dense_14 (Dense)             (None, 50)                4050      
_________________________________________________________________
dense_15 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_16 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_17 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 6         
Total params: 7,981
Trainable params: 7,981
Non-trainable params: 0
____________________________________________________

In [74]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [81]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn_3.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

nn_3.save("AlphabetSoupCharity_Optimization2.h5")

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5565 - accuracy: 0.7311
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5565 - accuracy: 0.7302: 0s - loss: 0.5593 
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7308: 0s - loss: 0.5528 - accuracy: 0.73 - ETA: 0s - loss: 0.5533 
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5561 - accuracy: 0.7310

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7305
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7308: 0s - loss: 0.5536 - accu
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7304: 0s - loss: 0.5566 
Epoch 

804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7298: 0s - loss: 0.5602 - accuracy: 0. - ETA: 0s - loss: 0.5584 - accuracy: 
Epoch 55/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5559 - accuracy: 0.7294

Epoch 00055: saving model to checkpoints\weights.55.hdf5
Epoch 56/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7301: 0s - loss: 0.5471  - ETA: 0s - loss: 0.5486 - accu
Epoch 57/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7303: 0s - los - ETA: 0s - loss: 0.5559 - accuracy: 0.72
Epoch 58/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5568 - accuracy: 0.7306: 0s - loss: 0.5572 - accuracy: 0.73
Epoch 59/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7311
Epoch 60/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5558 - accuracy: 0.7314: 0s - loss: 0.5579 - accu

Ep

ValueError: in user code:

    C:\Users\kimbe\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py:1330 test_function  *
        return step_function(self, iterator)
    C:\Users\kimbe\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py:1320 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\kimbe\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\kimbe\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\kimbe\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\kimbe\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py:1313 

# # Optimization Attempt 3 - Add layers AND change activation functions

In [78]:
nn_4 = tf.keras.models.Sequential()

# First hidden layer
nn_4.add(tf.keras.layers.Dense(units=20, activation="sigmoid", input_dim=number_input_features))

# Second hidden layer
nn_4.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))

# third hidden layer
nn_4.add(tf.keras.layers.Dense(units=20, activation="sigmoid"))

# fourth hidden layer
nn_4.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# fifth hidden layer
nn_4.add(tf.keras.layers.Dense(units=5, activation="sigmoid"))

# Output layer
nn_4.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn_4.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 20)                660       
_________________________________________________________________
dense_26 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_27 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_28 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 3,001
Trainable params: 3,001
Non-trainable params: 0
____________________________________________________

In [79]:
# Compile the model
nn_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [82]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn_4.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

nn_4.save("AlphabetSoupCharity_Optimization3.h5")

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - loss: 8.1786 
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - loss: 8.2311 - accuracy: 0. - ETA: 0s - loss: 8.1
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - loss: 8.2820 - 
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
804/804 [==============================] - ETA: 0s - loss: 8.2081 - accuracy: 0.46 - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - loss: 8.1993 - accura
Epoch 8/100
804/804 [==============================] - 1s 1ms/step 

804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - loss: 8.1977 - accuracy: 0.46
Epoch 60/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679

Epoch 00060: saving model to checkpoints\weights.60.hdf5
Epoch 61/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - loss: 8.2013 - accuracy: 0.46
Epoch 62/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - loss: 8.1811 - accura - ETA: 0s - loss: 8.2032 - accuracy: 0.46
Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - los
Epoch 65/100
804/804 [==============================] - 1s 1ms/step - loss: 8.2084 - accuracy: 0.4679: 0s - l

Epoch 00065: saving model to checkpoints\weights.65.hdf5
Epoch 66/100
804/804 [=

# # Using a function to determine best model hyperparameters

In [38]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [39]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

In [40]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 23s]
val_accuracy: 0.7244315147399902

Best val_accuracy So Far: 0.7301457524299622
Total elapsed time: 00h 08m 37s
INFO:tensorflow:Oracle triggered exit


In [41]:
# Evaluate the top three models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5545 - accuracy: 0.7301
Loss: 0.5544745326042175, Accuracy: 0.7301457524299622
268/268 - 0s - loss: 0.5609 - accuracy: 0.7276
Loss: 0.5609431266784668, Accuracy: 0.727580189704895
268/268 - 0s - loss: 0.5560 - accuracy: 0.7272
Loss: 0.5559980869293213, Accuracy: 0.7272303104400635


In [42]:
# Get  best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(2)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 7,
 'num_layers': 5,
 'units_0': 5,
 'units_1': 5,
 'units_2': 7,
 'units_3': 3,
 'units_4': 7,
 'units_5': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}